In [2]:
import pandas as pd
import numpy as np

from math import sqrt
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error

In [3]:
df_links    = pd.read_csv('./datasets/links.csv')
df_movies   = pd.read_csv('./datasets/movies.csv')
df_ratings  = pd.read_csv('./datasets/ratings.csv')
df_tags     = pd.read_csv('./datasets/tags.csv')

In [4]:
user_item_matrix = df_ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_item_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
user_item_matrix = user_item_matrix.astype(float)  

user_item_matrix_sparse = csr_matrix(user_item_matrix)

U, sigma, Vt = svds(user_item_matrix_sparse, k=50)
sigma = np.diag(sigma)

In [6]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [7]:
min_rating = np.min(predicted_ratings)
max_rating = np.max(predicted_ratings)
scaled_predicted_ratings = 5.0 * (predicted_ratings - min_rating) / (max_rating - min_rating)

In [8]:
scaled_predicted_ratings_df = pd.DataFrame(scaled_predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

In [9]:
def recommend_movies_svd(user_id, top_n=10):
    user_ratings = user_item_matrix.loc[user_id]
    user_predicted_ratings = scaled_predicted_ratings_df.loc[user_id].drop(user_ratings[user_ratings > 0].index)
    top_n_movie_ids = user_predicted_ratings.sort_values(ascending=False).head(top_n).index
    recommended_movies = df_movies[df_movies['movieId'].isin(top_n_movie_ids)]
    return recommended_movies

In [10]:
actual = user_item_matrix.values.flatten()
predicted = scaled_predicted_ratings_df.values.flatten()

In [11]:
mask = actual > 0
actual_filtered = actual[mask]
predicted_filtered = predicted[mask]

In [12]:
rmse = sqrt(mean_squared_error(actual_filtered, predicted_filtered))
print(f'RMSE: {rmse}')

RMSE: 1.6018934304006889


In [13]:
recommend_movies_svd(2,10)

,movieId,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller
3638,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
4800,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
6331,48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
7043,69122,"Hangover, The (2009)",Comedy|Crime
7413,80463,"Social Network, The (2010)",Drama
8879,134130,The Martian (2015),Adventure|Drama|Sci-Fi


In [14]:
actual_filtered

array([4., 4., 4., ..., 5., 5., 3.])

In [15]:
predicted_filtered

array([2.27302275, 1.76060002, 2.40260353, ..., 3.30600736, 3.29777484,
       2.54193563])